In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
from datetime import datetime, timedelta
import pandas as pd

#current_date = pd.to_datetime(datetime.utcnow()).floor('H') # - timedelta(hours=1)
current_date = pd.to_datetime('2024-10-30 23:00:00')
# current_date = pd.to_datetime('2024-10-31 00:00:00')
print(f'{current_date=}')
# current_date = pd.Timestamp('2023-02-28 09:00:00')

current_date=Timestamp('2024-10-30 23:00:00')


In [10]:
from src.inference import load_batch_of_features_from_store

features = load_batch_of_features_from_store(current_date)

2024-12-23 14:52:47,678 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-23 14:52:47,681 INFO: Initializing external client
2024-12-23 14:52:47,681 INFO: Base URL: https://c.app.hopsworks.ai:443


2024-12-23 14:52:48,615 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104
2024-12-23 14:52:50,704 INFO: Feature view already exists, skipping creation.
2024-12-23 14:52:50,704 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-23 14:52:50,707 INFO: Initializing external client
2024-12-23 14:52:50,708 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-23 14:52:51,692 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.54s) 


In [11]:
from src.inference import (
    load_model_from_registry,
    get_model_predictions
)

model = load_model_from_registry()
predictions = get_model_predictions(model, features)

2024-12-23 14:52:59,391 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-23 14:52:59,393 INFO: Initializing external client
2024-12-23 14:52:59,394 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-23 14:53:00,406 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104
[LightGBM] [Warning] feature_fraction is set=0.9904213488145986, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9904213488145986
[LightGBM] [Warning] bagging_fraction is set=0.9938247583773121, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9938247583773121


In [12]:
predictions['pickup_hour'] = current_date
predictions

,pickup_location_id,predicted_demand,pickup_hour
0,1,11.0,2024-10-30 23:00:00
1,2,9.0,2024-10-30 23:00:00
2,3,9.0,2024-10-30 23:00:00
3,4,16.0,2024-10-30 23:00:00
4,6,9.0,2024-10-30 23:00:00
...,...,...,...
250,261,9.0,2024-10-30 23:00:00
251,262,10.0,2024-10-30 23:00:00
252,263,9.0,2024-10-30 23:00:00
253,264,9.0,2024-10-30 23:00:00


In [13]:
from src.feature_store_api import get_feature_store
import src.config as config

# connect to the feature group
feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTIONS,
    version=1,
    description="Predictions generate by our production model",
    primary_key = ['pickup_location_id', 'pickup_hour'],
    event_time='pickup_hour',
)

2024-12-23 14:53:07,193 INFO: Closing external client and cleaning up certificates.
Connection closed.
2024-12-23 14:53:07,197 INFO: Initializing external client
2024-12-23 14:53:07,197 INFO: Base URL: https://c.app.hopsworks.ai:443
2024-12-23 14:53:08,277 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1192104


In [14]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1192104/fs/1181777/fg/1394145


Uploading Dataframe: 100.00% |██████████| Rows 255/255 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: model_predictions_feature_group_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1192104/jobs/named/model_predictions_feature_group_1_offline_fg_materialization/executions


(Job('model_predictions_feature_group_1_offline_fg_materialization', 'SPARK'),
 None)